## Crear Dataset en el formato necesario

In [4]:
import os
import shutil

from_folder = "SoccerNet/calibration"
to_folder = "dataset/images"

# Crear la carpeta destino si no existe
os.makedirs(to_folder, exist_ok=True)

# Iterar sobre cada subcarpeta
dirs = os.listdir(from_folder)
for dir_name in dirs:
    subfolder_path = os.path.join(from_folder, dir_name)
    to_folder_path = os.path.join(to_folder, dir_name)
    if not os.path.isdir(subfolder_path):
        continue  # Saltar si no es carpeta
    
    if not os.path.exists(to_folder_path):
        os.makedirs(to_folder_path)

    # Listar todas las imágenes en la subcarpeta
    images = [img for img in os.listdir(subfolder_path) if img.endswith(".jpg") or img.endswith(".png")]

    # Copiar las imágenes a la carpeta destino
    for img in images:
        src_path = os.path.join(subfolder_path, img)
        dst_path = os.path.join(to_folder_path, img)
        shutil.copy(src_path, dst_path)

print("Imágenes copiadas correctamente.")

Imágenes copiadas correctamente.


In [ ]:
import os
import sys
import json
from_folder = "SoccerNet/calibration"
labels_folder = "dataset/labels"

# Diccionario para mapear etiquetas a IDs
class_mapping = {
    "Goal right crossbar": 0,
    "Goal right post left": 0,
    "Goal right post right": 0,
    "Goal left post left": 0,
    "Goal left post left ": 0,
    "Goal left post left": 0,
    "Goal left post right": 0,
    "Goal left crossbar": 0,
    "Small rect. right main": 1,
    "Small rect. right top": 1,
    "Side line right": 1,
    "Side line top": 1,
    "Side line bottom": 1,
    "Big rect. right top": 1,
    "Big rect. right main": 1,
    "Circle right": 2,
    "Side line left": 1,
    "Big rect. left top": 1,
    "Big rect. left bottom": 1,
    "Big rect. left main": 1,
    "Big rect. right bottom": 1,
    "Small rect. right bottom": 1,
    "Small rect. left top": 1,
    "Small rect. left bottom": 1,
    "Small rect. left main": 1,
    "Circle left": 2,
    "Circle central": 3,
    "Middle line": 1,
    "Line unknown": 4,
    "Goal unknown": 0,
}

# Función auxiliar para obtener el bounding box
def get_bbox_from_points(points):
    xs = [p["x"] for p in points]
    ys = [p["y"] for p in points]
    x_min, x_max = min(xs), max(xs)
    y_min, y_max = min(ys), max(ys)
    x_center = (x_min + x_max) / 2
    y_center = (y_min + y_max) / 2
    width = x_max - x_min
    height = y_max - y_min
    return x_center, y_center, width, height

# Crear la carpeta destino si no existe
os.makedirs(labels_folder, exist_ok=True)

# Iterar sobre cada subcarpeta
dirs = os.listdir(from_folder)
for dir_name in dirs:
    subfolder_path = os.path.join(from_folder, dir_name)
    to_folder_path = os.path.join(labels_folder, dir_name)
    if not os.path.isdir(subfolder_path):
        continue  # Saltar si no es carpeta
    
    if not os.path.exists(to_folder_path):
        os.makedirs(to_folder_path)

    #listar todos los json
    jsons = [json for json in os.listdir(subfolder_path) if json.endswith(".json")]

    for j in jsons:
        #si el nombre del archivo no es numérico, saltar
        if not j[:-5].isdigit():
            print(f"Nombre de archivo no numérico: {j}")
            continue
        with open(os.path.join(subfolder_path, j), "r") as f:
            try:
                data = json.load(f)
            except Exception as e:
                print(f"Error al cargar el archivo JSON {j}: {e}")
                continue
            label_txt = os.path.join(to_folder_path, j.replace(".json", ".txt"))
            with open(label_txt, "w") as label_file:
                for label, points in data.items():
                    if label not in class_mapping:
                        print(f"Etiqueta desconocida: {label} en {j}")
                    if label not in class_mapping or len(points) < 2:
                        continue
                    class_id = class_mapping[label]
                    x_c, y_c, w, h = get_bbox_from_points(points)
                    label_file.write(f"{class_id} {x_c:.6f} {y_c:.6f} {w:.6f} {h:.6f}\n")
print("Etiquetas generadas correctamente.")

Nombre de archivo no numérico: match_info.json
Nombre de archivo no numérico: match_info.json
Nombre de archivo no numérico: match_info.json
Etiquetas generadas correctamente.


# Finetunear Modelo

In [20]:
from ultralytics import YOLO

# Cargar un modelo base
model = YOLO('yolov8n.pt')  
# Entrenar (fine-tune) con tus datos
model.train(data='line_dataset.yaml', epochs=50, imgsz=640, batch=16)

Ultralytics 8.3.141 🚀 Python-3.12.3 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060, 7911MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=line_dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrai

100%|██████████| 755k/755k [00:01<00:00, 743kB/s]

Overriding model.yaml nc=80 with nc=5

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

Model summary: 129 layers, 3,011,823 parameters, 3,011,807 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 5662.9±1413.8 MB/s, size: 162.6 KB)


train: Scanning /home/dani/Desktop/futbol/dataset/labels/train... 14513 images, 0 backgrounds, 379 corrupt: 100%|██████████| 14513/14513 [00:04<00:00, 3346.92it/s]

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      41/50      2.64G     0.6582      0.436     0.8766         44        640: 100%|██████████| 884/884 [02:48<00:00,  5.24it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 85/85 [00:35<00:00,  2.37it/s]

                   all       2714      26158      0.931      0.607      0.655      0.557

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size



      42/50      2.64G     0.6457     0.4218     0.8726        119        640:  87%|████████▋ | 768/884 [01:52<00:12,  9.10it/s]Exception in thread Thread-72 (_pin_memory_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/home/dani/Desktop/futbol/.venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/home/dani/Desktop/futbol/.venv/lib/python3.12/site-packages/torch/utils/data/_utils/pin_memory.py", line 61, in _pin_memory_loop
    do_one_step()
  File "/home/dani/Desktop/futbol/.venv/lib/python3.12/site-packages/torch/utils/data/_utils/pin_memory.py", line 37, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
      42/50      2.64G     0.6457     0.4218     0.8726        119        640:  87%|████████▋ | 

KeyboardInterrupt: 

In [21]:
#guardar el modelo
model.save("yolov8n_line.pt")

In [ ]:
from ultralytics import YOLO
import cv2

#cargar el modelo
model = YOLO("yolov8n_line.pt")
# Realizar inferencia a la imagen example.jpg
results = model.predict(source="example.jpg", save=True, save_txt=True, project="dataset/predictions", name="example")
# Mostrar resultados
for result in results:
    # Mostrar la imagen con las detecciones
    cv2.imshow("Detecciones", result.plot())
    cv2.waitKey(0)  # Esperar a que se presione una tecla
    cv2.destroyAllWindows()


image 1/1 /home/dani/Desktop/futbol/example.jpg: 416x640 2 field_lines, 11.2ms
Speed: 2.1ms preprocess, 11.2ms inference, 14.3ms postprocess per image at shape (1, 3, 416, 640)
Results saved to dataset/predictions/example2
1 label saved to dataset/predictions/example2/labels
